# PowerShell Compare-Object Cmdlet
What: How do we do comparisons of objects/properties in PowerShell?

- Comparing simple properties/items
- Can we compare object properties from differnet objects?
- Can we compare an object to file contents?


First, let's look at the parameters of the cmdlet `Get-Help Compare-Object`.

In [2]:
Get-Help Compare-Object


NAME
    Compare-Object
    
SYNOPSIS
    Compares two sets of objects.
    
    
SYNTAX
    Compare-Object [-ReferenceObject] <System.Management.Automation.PSObject[]> 
    [-DifferenceObject] <System.Management.Automation.PSObject[]> [-CaseSensitive] [-Culture 
    <System.String>] [-ExcludeDifferent] [-IncludeEqual] [-PassThru] [-Property <System.Object[]>] 
    [-SyncWindow <System.Int32>] [<CommonParameters>]
    
    
DESCRIPTION
    The `Compare-Object` cmdlet compares two sets of objects. One set of objects is the reference 
    , and the other set of objects is the difference .
    
    `Compare-Object` checks for available methods of comparing a whole object. If it can't find a 
    suitable method, it calls the ToString() methods of the input objects and compares the string 
    results. You can provide one or more properties to be used for comparison. When properties are 
    provided, the cmdlet compares the values of those properties only.
    
    The result of the comp

Pretty easy to follow, Compares two sets of objects.  One object is the reference, the other object is the difference.

Keep an eye on that second paragraph, we'll talk about that again.

`Compare-Object` checks for available methods of comparing a whole object. If it can't find a 
    suitable method, it calls the ToString() methods of the input objects and compares the string 
    results.

In [14]:
get-content -Path "Store1.txt"


Apples
Oranges
Bananas
Peaches
Lemons
Limes


In [15]:
get-content -Path "Store2.txt"

Apples
Oranges
Lemons
Limes
Grapes
Tomatoes


In [16]:
Compare-Object -ReferenceObject (get-content -Path "Store1.txt") -DifferenceObject (get-content -Path "Store2.txt")


InputObject SideIndicator
----------- -------------
Grapes      =>
Tomatoes    =>
Bananas     <=
Peaches     <=



The "=>" SideIndicator is showing that these items are in the Difference Object (Store2), and not in the Reference Object (Store 1)

The "<=" SideIndicator is showing the opposite, so these items are in the Reference Object (Store 1) but not in the Different Object (Store 2)

In [17]:
Compare-Object -ReferenceObject (get-content -Path "Store1.txt") -DifferenceObject (get-content -Path "Store2.txt") -ExcludeDifferent


InputObject SideIndicator
----------- -------------
Apples      ==
Oranges     ==
Lemons      ==
Limes       ==



Using the -ExcludeDifferent parameter shows only the items that are the same in both objects.  Notice the indicator is now a "==".
And, if there's a -ExcludeDifferent Parameter, there must be a -IncludeEqual parameter, right?

In [18]:
Compare-Object -ReferenceObject (get-content -Path "Store1.txt") -DifferenceObject (get-content -Path "Store2.txt") -IncludeEqual


InputObject SideIndicator
----------- -------------
Apples      ==
Oranges     ==
Lemons      ==
Limes       ==
Grapes      =>
Tomatoes    =>
Bananas     <=
Peaches     <=



Yep!  -IncludeEqual shows you the items that are not in each object, as well as the ones that are in each.

So how about comparing properties from different object types?  Do we dare?!  Sure, why not...

In [19]:
get-content -Path "./FileNames.txt"

Get-ServiceUsingParameterFile.ps1
LICENSE
README.md
ScriptParameters.ps1
services.csv
services.json
PowerShellParameterBindingExploration.ipynb
Test-ValueFromPipelineBehavior_withParamSet.ps1
ObjectPropertyFormatChange.ipynb
ObjectPropertyFormatting.ipynb
ArrayOfStrings.json
Test-JSONInput-noName.ps1


In [7]:
Compare-Object -ReferenceObject (Get-ChildItem -File -Path /Users/V0X9585/AutomationEducation/) `
-DifferenceObject (get-content -Path "/Users/V0X9585/AutomationEducation/Compare-Object/FileNames.txt") -IncludeEqual


InputObject                                                          SideIndicator
-----------                                                          -------------
Get-ServiceUsingParameterFile.ps1                                    =>
LICENSE                                                              =>
README.md                                                            =>
ScriptParameters.ps1                                                 =>
services.csv                                                         =>
Store1.txt                                                           =>
services.json                                                        =>
PowerShellParameterBindingExploration.ipynb                          =>
Test-ValueFromPipelineBehavior_withParamSet.ps1                      =>
ObjectPropertyFormatChange.ipynb                                     =>
ObjectPropertyFormatting.ipynb                                       =>
ArrayOfStrings.json                      

That didn't seem to work like we expected?

Remmeber when Get-Help told us that Compare-Object will try to find a method on the object to use in the comparision, but will execute the ToString method on the object if it can't?

In [23]:
(Get-ChildItem -File) | ForEach-Object{$_.ToString()} 

/Users/V0X9585/AutomationEducation/Compare-Object/Compare-Object-Examples.ipynb
/Users/V0X9585/AutomationEducation/Compare-Object/FileNames.txt
/Users/V0X9585/AutomationEducation/Compare-Object/Store1.txt
/Users/V0X9585/AutomationEducation/Compare-Object/Store2.txt


So we have a string from Get-ChildItem of the filename with path, and we're trying to compare it to just the filename from our text file.
In this case, we need to make sure the -ReferenceObject is the same as what we are looking for.  So we tell Get-ChildItem to give us just the Name of the file in the cmdlet's output.

In [8]:
Compare-Object -ReferenceObject (Get-Childitem -File -Path "/Users/V0X9585/AutomationEducation/" -recurse| `
Select-Object -expand Name) -DifferenceObject (get-content -Path "/Users/V0X9585/AutomationEducation/Compare-Object/FileNames.txt") -IncludeEqual


InputObject                                     SideIndicator
-----------                                     -------------
Get-ServiceUsingParameterFile.ps1               ==
LICENSE                                         ==
README.md                                       ==
ScriptParameters.ps1                            ==
services.csv                                    ==
services.json                                   ==
PowerShellParameterBindingExploration.ipynb     ==
Test-ValueFromPipelineBehavior_withParamSet.ps1 ==
Store1.txt                                      ==
ObjectPropertyFormatChange.ipynb                ==
ObjectPropertyFormatting.ipynb                  ==
ArrayOfStrings.json                             ==
Test-JSONInput-noName.ps1                       ==
Test-ValueFromPipelineBehavior.ps1              <=
Calculated-Properties.ps1                       <=
All-FileNames-T2.csv                            <=
All-FileNames.csv                               <=
All-File

And there we go.  The SideIndicator of "==" shows us the files that we found in the directories that match what we have in our supplied text file.

Huzzah!!

Now let's look at comparing object properties by Parameter!

In [11]:
Import-CSV -Path ./All-FileNames.csv | select-object -first 1


PSPath              : Microsoft.PowerShell.Core\FileSystem::/Users/V0X9585/AutomationEducation/Get-
                      ServiceUsingParameterFile.ps1
PSParentPath        : Microsoft.PowerShell.Core\FileSystem::/Users/V0X9585/AutomationEducation
PSChildName         : Get-ServiceUsingParameterFile.ps1
UnixStat            : System.Management.Automation.Platform+Unix+CommonStat
PSDrive             : /
PSProvider          : Microsoft.PowerShell.Core\FileSystem
PSIsContainer       : False
UnixMode            : -rw-r--r--
User                : V0X9585
Group               : staff
Size                : 242
Mode                : -----
ModeWithoutHardLink : -----
VersionInfo         : File:             
                      /Users/V0X9585/AutomationEducation/Get-ServiceUsingParameterFile.ps1
                      InternalName:     
                      OriginalFilename: 
                      FileVersion:      
                      FileDescription:  
                      Product:          

In [1]:
Compare-Object -ReferenceObject (import-csv ./All-FileNames.csv) -DifferenceObject (import-csv ./All-FileNames-T2.csv) -Property Basename


Basename                                    SideIndicator
--------                                    -------------
Test-ValueFromPipelineBehavior_withParamSet <=
Test-ValueFromPipelineBehavior              <=
Calculated-Properties                       <=
All-FileNames                               <=
All-FileNames                               <=
Compare-Object-Examples                     <=
FileNames                                   <=
Store1                                      <=
Store2                                      <=



In [2]:
Compare-Object -ReferenceObject (import-csv ./All-FileNames.csv) -DifferenceObject (import-csv ./All-FileNames-T2.csv) -Property Basename,Attributes


Basename                                    Attributes SideIndicator
--------                                    ---------- -------------
PowerShellParameterBindingExploration       AbbyNormal =>
ObjectPropertyFormatChange                  AbbyNormal =>
PowerShellParameterBindingExploration       Normal     <=
Test-ValueFromPipelineBehavior_withParamSet Normal     <=
Test-ValueFromPipelineBehavior              Normal     <=
Calculated-Properties                       Normal     <=
All-FileNames                               Normal     <=
All-FileNames                               Normal     <=
Compare-Object-Examples                     Normal     <=
FileNames                                   Normal     <=
Store1                                      Normal     <=
Store2                                      Normal     <=
ObjectPropertyFormatChange                  Normal     <=

